In [ ]:
from zipline.sources.TEJ_Api_Data import get_universe
import pandas as pd
import numpy as np
import TejToolAPI

def select(first,last,st,en,year):
    # 1. 取得股票池
    pool = get_universe(start=first, end=last, mkt_bd_e='TSE',
                        stktp_e='Common Stock', main_ind_e='General Industry')

    # 2. 取得年初與年末資料
    data_start = TejToolAPI.get_history_data(start=pd.Timestamp(st),end=pd.Timestamp(st),
                          ticker=pool,fin_type='A',
                          columns=['收盤價'],transfer_to_chinese=True)
    data_end = TejToolAPI.get_history_data( start=pd.Timestamp(en),end=pd.Timestamp(en),
                          ticker=pool,fin_type='A',
                          columns=['收盤價','股價淨值比','mktcap'],
                          transfer_to_chinese=True)
    # 3. 存檔年末資料
    data_end.to_excel(f"{year}.xlsx", index=False)

    # 4. 設 index
    data_start.set_index('股票代碼', inplace=True)
    data_end.set_index('股票代碼', inplace=True)

    # 5. 取交集
    coid = sorted(list(set(data_start.index) & set(data_end.index)))

    # 6. 建 DataFrame
    df = pd.DataFrame({
        '股票代碼': coid,
        '01收盤價': [data_start.loc[i,'收盤價'] for i in coid],
        '12收盤價': [data_end.loc[i,'收盤價'] for i in coid],
        'Return': [data_end.loc[i,'收盤價']/data_start.loc[i,'收盤價']-1 for i in coid],
        'pb': [data_end.loc[i,'股價淨值比'] for i in coid],
        'bm': [1/data_end.loc[i,'股價淨值比'] for i in coid],
        'mkt': [data_end.loc[i,'個股市值_元'] for i in coid]
    })

    # 7. 篩選函式
    def filter_stocks(percent1, percent2):
        df_mv = df.nsmallest(int(len(df)*percent1/100), 'mkt')['股票代碼']
        df_bm = df.nlargest(int(len(df)*percent1/100), 'bm')['股票代碼']
        df_ret = df.nlargest(int(len(df)*percent2/100), 'Return')['股票代碼']
        selected = sorted(list(set(df_mv) & set(df_bm) & set(df_ret)))
        print(selected)
        print(f"總共有 {len(selected)} 間公司")
        return selected

    selected = filter_stocks(30,30)
    return df, selected
